### Importing a bunch of stuff (most of it is useless here but i was lazy)

In [ ]:
import os
import glob
import h5py
import shutil
import imgaug as aug
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.image as mimg
import imgaug.augmenters as iaa
from os import listdir, makedirs, getcwd, remove
from os.path import isfile, join, abspath, exists, isdir, expanduser
from PIL import Image
from pathlib import Path
from skimage.io import imread
from skimage.transform import resize
from keras.models import Sequential, Model, load_model
from keras import applications
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.preprocessing.image import ImageDataGenerator,load_img, img_to_array
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten, SeparableConv2D
from keras.layers import GlobalMaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.merge import Concatenate
from keras.models import Model
from keras.optimizers import Adam, SGD, RMSprop
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from mlxtend.plotting import plot_confusion_matrix
from sklearn.metrics import confusion_matrix
import cv2
from keras import backend as K
import tensorflow as tf


# Disable multi-threading in tensorflow ops
session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
# Set the random seed in tensorflow at graph level
tf.compat.v1.set_random_seed(111)
# Define a tensorflow session with above session configs
sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
# Set the session in keras
tf.compat.v1.keras.backend.set_session(sess)

### Creating an empty model

In [ ]:
def create_empty_model():
    model = applications.inception_resnet_v2.InceptionResNetV2(
        include_top=False, #Default:(299,299,3)
        weights='imagenet',
        input_shape=(299,299,3),
        pooling='max'
    )

    # Freeze layers
    for layer in model.layers:
        layer.trainable = False

    # Add trainable layers to the model
    x = model.output
    #model.summary()
    predictions = Dense(2, activation='softmax')(x)

    # Create the final model and compile it
    final_model = Model(inputs=model.input, outputs=predictions)

    # Compile model with optimization setting
    opt = Adam(lr=0.001, decay=1e-5)
    final_model.compile(loss='binary_crossentropy', metrics=['accuracy'],optimizer=opt)

    return final_model

### Loading in the best models for normal vs pneumonia and bacterial vs viral

In [ ]:
normpneum_test_model = create_empty_model()
normpneum_test_model.load_weights('best_normpneum_checkpoint.hdf5') #TODO: change to latest best model
bactviral_test_model = create_empty_model()
bactviral_test_model.load_weights('best_bactviral_checkpoint.hdf5') #TODO: change to latest best model


### Loading in images

In [ ]:
bacteria_path = Path("chest_xray/images/test/BACTERIA")
virus_path = Path("chest_xray/images/test/VIRUS")
normal_path = Path("chest_xray/images/test/NORMAL")

In [ ]:
bacteria_filenames = []
virus_filenames = []
normal_filenames = []

for (dirpath, dirnames, filenames) in os.walk(bacteria_path):
    bacteria_filenames += [os.path.join(dirpath, file) for file in filenames]
for (dirpath, dirnames, filenames) in os.walk(virus_path):
    virus_filenames += [os.path.join(dirpath, file) for file in filenames]
for (dirpath, dirnames, filenames) in os.walk(normal_path):
    normal_filenames += [os.path.join(dirpath, file) for file in filenames]

### Select some images to show

In [ ]:
#Now just choosing random idices, would be better if we know how they are classified by our model

import random
normal_index = random.randrange(len(normal_filenames))
norm_img_filename = normal_filenames[normal_index]
norm_image = np.array(load_img(norm_img_filename), dtype=np.uint8)

bacteria_index = random.randrange(len(bacteria_filenames))
bac_img_filename = bacteria_filenames[bacteria_index]
bac_image = np.array(load_img(bac_img_filename), dtype=np.uint8)


virus_index = random.randrange(len(virus_filenames))
vir_img_filename = virus_filenames[virus_index]
virus_image = np.array(load_img(vir_img_filename), dtype=np.uint8)

f, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(20, 20))
ax1.imshow(norm_image)
ax1.set_title('Normal')
ax2.imshow(bac_image)
ax2.set_title('Bacteria')
ax3.imshow(virus_image)
ax3.set_title('Virus')

### Final layer of the models (we should add our own conv layer)

In [ ]:
normpneum_final_layer = 'conv_7b'
bactviral_final_layer = 'conv_7b'

normpneum_test_model.summary()
#bactviral_test_model.summary()

In [ ]:
from gradcamutils import GradCam, GradCamPlusPlus, ScoreCam, build_guided_model, GuidedBackPropagation, superimpose, read_and_preprocess_img

In [ ]:
norm_img_array = read_and_preprocess_img(norm_img_filename, size=(299,299))
bac_img_array = read_and_preprocess_img(bac_img_filename, size=(299,299))
vir_img_array = read_and_preprocess_img(vir_img_filename, size=(299,299))

In [ ]:
maxN = 100 
# maxN makes the ScoreCam below way faster, 
# but we do not really know yet what it does (and it can affect the final image drastically)

### With normal vs pneumonia model:

In [ ]:
norm_score_cam = ScoreCam(normpneum_test_model, norm_img_array, normpneum_final_layer, max_N=maxN)

In [ ]:
bac_score_cam = ScoreCam(normpneum_test_model, bac_img_array, normpneum_final_layer, max_N=maxN)

In [ ]:
vir_score_cam = ScoreCam(normpneum_test_model, vir_img_array, normpneum_final_layer, max_N=maxN)

In [ ]:
norm_superimposed = superimpose(norm_img_filename, norm_score_cam)
bac_superimposed = superimpose(bac_img_filename, bac_score_cam)
vir_superimposed = superimpose(vir_img_filename, vir_score_cam)

f, axarr = plt.subplots(2, 3, figsize=(15,10))
plt.tight_layout()
axarr[0, 0].imshow(norm_score_cam)
axarr[0, 0].set_title('Normal')
axarr[0, 1].imshow(bac_score_cam)
axarr[0, 1].set_title('Bacterial')
axarr[0, 2].imshow(vir_score_cam)
axarr[0, 2].set_title('Bacterial')


axarr[1, 0].imshow(norm_superimposed)
axarr[1, 1].imshow(bac_superimposed)
axarr[1, 2].imshow(vir_superimposed)

### With bacterial vs viral model:

In [ ]:
norm_score_cam2 = ScoreCam(bactviral_test_model, norm_img_array, normpneum_final_layer, max_N=maxN)

In [ ]:
bac_score_cam2 = ScoreCam(bactviral_test_model, bac_img_array, normpneum_final_layer, max_N=maxN)

In [ ]:
vir_score_cam2 = ScoreCam(bactviral_test_model, vir_img_array, normpneum_final_layer, max_N=maxN)

In [ ]:
norm_superimposed2 = superimpose(norm_img_filename, norm_score_cam2)
bac_superimposed2 = superimpose(bac_img_filename, bac_score_cam2)
vir_superimposed2 = superimpose(vir_img_filename, vir_score_cam2)

f, axarr = plt.subplots(2, 3, figsize=(15,10))
axarr[0, 0].imshow(norm_score_cam2)
axarr[0, 0].set_title('Normal')
axarr[0, 1].imshow(bac_score_cam2)
axarr[0, 1].set_title('Bacterial')
axarr[0, 2].imshow(vir_score_cam2)
axarr[0, 2].set_title('Bacterial')


axarr[1, 0].imshow(norm_superimposed2)
axarr[1, 1].imshow(bac_superimposed2)
axarr[1, 2].imshow(vir_superimposed2)